<a href="https://colab.research.google.com/github/Arul-Vincy-Mary/nerd/blob/main/Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyLDAvis

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import email

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import ne_chunk

nltk.download('stopwords')
nltk.download('punkt')

# spacy for lemmatization
import spacy
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser

# for plotting
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import ne_chunk


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
# spacy for lemmatization
import spacy
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser

In [ ]:
# for plotting
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Dataset/emails.csv')

In [ ]:
e = email.message_from_string(data.message[2054])
e.get_payload()

'Phillip;\n\nThis attachment is my start on your 1031 search.  From my experience, you\nwill probably need to park the money until we find the right investment for\nyou.  I have not had time to analize all of the possibilities.\n\nAre you coming on Friday? Do you want me to drive by these properties\ntomorrow, or are we going to do it Friday?\n\nPlease reply\n\n\n\n - Phillip Allen 1031 Spreadsheet.xls '

In [ ]:
e.items()

In [ ]:
e.get_payload()

In [ ]:
df = data.copy(deep=True)

In [ ]:
def get_field(field, messages):
    column = []
    for message in messages:
        e = email.message_from_string(message)
        column.append(e.get(field))
    return column

In [ ]:
df['Subject'] = get_field("Subject", df['message'])
df['Folder'] = get_field("X-Folder", df['message'])
df['From'] = get_field("X-From", df['message'])
df['To'] = get_field("X-To", df['message'])

In [ ]:
def body(messages):
    column = []
    for message in messages:
        e = email.message_from_string(message)
        column.append(e.get_payload())
    return column

In [ ]:
df['body'] = body(df['message'])

In [ ]:
df.head()

,file,message,Subject,Folder,From,To,body
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,Here is our forecast\n\n
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Re:,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,Re: test,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Phillip K Allen,Leah Van Arsdall,test successful. way to go!!!
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Phillip K Allen,Randall L Gay,"Randy,\n\n Can you send me a schedule of the s..."
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Re: Hello,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Phillip K Allen,Greg Piper,Let's shoot for Tuesday at 11:45.


In [ ]:
col = ['file','message']
df = df.drop(columns = col, axis = 1)

In [ ]:
print(df.shape)

(517401, 5)


In [ ]:
def preprocess_folder(folders):
    column = []
    for folder in folders:
        if (folder is None or folder == ""):
            column.append(np.nan)
        else:
            column.append(folder.split("\\")[-1].lower())
    return column

In [ ]:
df['Folder'] = preprocess_folder(df['Folder'])

In [ ]:
df.head(10)

,Subject,Folder,From,To,body
0,,'sent mail,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,Here is our forecast\n\n
1,Re:,'sent mail,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,Traveling to have a business meeting takes the...
2,Re: test,'sent mail,Phillip K Allen,Leah Van Arsdall,test successful. way to go!!!
3,,'sent mail,Phillip K Allen,Randall L Gay,"Randy,\n\n Can you send me a schedule of the s..."
4,Re: Hello,'sent mail,Phillip K Allen,Greg Piper,Let's shoot for Tuesday at 11:45.
5,Re: Hello,'sent mail,Phillip K Allen,Greg Piper,"Greg,\n\n How about either next Tuesday or Thu..."
6,,'sent mail,Phillip K Allen,"david.l.johnson@enron.com, John Shafer",Please cc the following distribution list with...
7,Re: PRC review - phone calls,'sent mail,Phillip K Allen,Joyce Teixeira,any morning between 10 and 11:30
8,Re: High Speed Internet Access,'sent mail,Phillip K Allen,Mark Scott,1. login: pallen pw: ke9davis\n\n I don't thi...
9,FW: fixed forward or other Collar floor gas pr...,'sent mail,Phillip K Allen,zimam@enron.com,---------------------- Forwarded by Phillip K ...


In [ ]:
df_try = df.copy(deep=True)

In [ ]:
new = df_try['Folder'].value_counts()

In [ ]:
new.head(35)

all documents          128103
discussion threads      58609
sent                    58168
deleted items           50987
inbox                   41507
sent items              37920
notes inbox             36665
'sent mail              30237
untitled                 8957
personal                 2642
attachments              2026
meetings                 1872
calendar                 1732
schedule crawler         1398
logistics                1192
tw-commercial group      1159
california               1052
bill williams iii        1022
deal discrepancies        897
management                802
esvl                      715
e-mail bin                614
resumes                   609
tufco                     609
ces                       576
online trading            573
junk                      560
junk file                 511
ooc                       511
genco-jv_ipo              486
projects                  468
archives                  448
contacts                  437
corporate 

In [ ]:
df_logistics = df_try[df_try.Folder=='logistics']
df_meetings = df_try[df_try.Folder=='meetings']
df_online_trade = df_try[df_try.Folder=='online trading']
df_corporate = df_try[df_try.Folder=='corporate']
df_projects = df_try[df_try.Folder=='projects']

df_sent = df_try[df_try.Folder=='sent']
df_sent = df_sent.sample(n=5000)

df_sent_mail = df_try[df_try.Folder=="'sent mail"]
df_sent_mail = df_sent_mail.sample(n=1000)

In [ ]:
df_meetings = df_try[df_try.Folder=='meetings']

In [ ]:
df_online_trade = df_try[df_try.Folder=='online trading']

In [ ]:
df_corporate = df_try[df_try.Folder=='corporate']

In [ ]:
df_projects = df_try[df_try.Folder=='projects']

In [ ]:
df_sent = df_try[df_try.Folder=='sent']
df_sent = df_sent.sample(n=5000)

In [ ]:
df_sent_mail = df_try[df_try.Folder=="'sent mail"]
df_sent_mail = df_sent_mail.sample(n=1000)

In [ ]:
def preprocess_body(body):
  column = []
  for file in body:
    column.append(file.lower())
  return column

In [ ]:
def remove_forwarded_message(body):
  column = []
  for file in body:
   pattern = re.compile(r'[- ]*forwarded by[\S\s]*subject:[\S\t ]*')
   column.append(re.sub(pattern,'',file))
  return column

In [ ]:
def remove_email(body):
  column = []
  for file in body:
   pattern = re.compile(r'[a-zA-Z0-9._+-]+@[a-zA-Z0-9]+\.[a-zA-Z.]+')
   column.append(re.sub(pattern,' ',file))
  return column
  

In [ ]:
def remove_url(body):
  column = []
  for file in body:
   pattern = re.compile(r'(https?:)//(www.)?[a-zA-Z0-9.]+\.[a-zA-Z0-9.]+')
   column.append(re.sub(pattern,' ',file))
  return column
  

In [ ]:
def remove_repeating_words(body):
  column = []
  for file in body:
   pattern = re.compile(r'[a-zA-Z]+(@|\.)[a-zA-Z.]+')
   column.append(re.sub(pattern,' ',file))
  return column
  

In [ ]:
def change_contracted_words(body):
  column = []
  Apos_dict={"'s":" is","n't":" not","'m":" am","'ll":" will",
             "'d":" would","'ve":" have","'re":" are"}
  
  for file in body:
    for key,value in Apos_dict.items():
      if key in file:
        file = file.replace(key,value)
    column.append(file)
  
  return column

In [ ]:
def remove_spec_characters(body):
  column = []
  for file in body:
   pattern = re.compile(r'(^\w\s)|(\W)')
   column.append(re.sub(pattern,' ',file))
  return column
  

In [ ]:
def remove_num(body):
  column = []
  for file in body:
   pattern = re.compile(r'(\d)')
   column.append(re.sub(pattern,' ',file))
  return column

In [ ]:
def remove_extra_spaces(body):
  column = []
  for file in body:
   column.append(re.sub('( +)|(_+)',' ',file))
  return column

In [ ]:
df_try['body'] = preprocess_body(df_try['body'])

In [ ]:
df_try['body'] = remove_forwarded_message(df_try['body'])

In [ ]:
df_try['body'] = remove_email(df_try['body'])

In [ ]:
df_try['body'] = remove_url(df_try['body'])

In [ ]:
df_try['body'] = remove_repeating_words(df_try['body'])

In [ ]:
df_try['body'] = change_contracted_words(df_try['body'])

In [ ]:
df_try['body'] = remove_spec_characters(df_try['body'])

In [ ]:
df_try['body'] = remove_num(df_try['body'])

In [ ]:
df_try['body'] = remove_extra_spaces(df_try['body'])

In [ ]:
df_try.head(10)

In [ ]:
df_try.isnull().sum()

In [ ]:
df_logistics['body'] = preprocess_body(df_logistics['body'])
df_logistics['body'] = remove_forwarded_message(df_logistics['body'])
df_logistics['body'] = remove_email(df_logistics['body'])
df_logistics['body'] = remove_url(df_logistics['body'])
df_logistics['body'] = remove_repeating_words(df_logistics['body'])
df_logistics['body'] = change_contracted_words(df_logistics['body'])
df_logistics['body'] = remove_spec_characters(df_logistics['body'])
df_logistics['body'] = remove_num(df_logistics['body'])
df_logistics['body'] = remove_extra_spaces(df_logistics['body'])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [ ]:
df_online_trade['body'] = preprocess_body(df_online_trade['body'])
df_online_trade['body'] = remove_forwarded_message(df_online_trade['body'])
df_online_trade['body'] = remove_email(df_online_trade['body'])
df_online_trade['body'] = remove_url(df_online_trade['body'])
df_online_trade['body'] = remove_repeating_words(df_online_trade['body'])
df_online_trade['body'] = change_contracted_words(df_online_trade['body'])
df_online_trade['body'] = remove_spec_characters(df_online_trade['body'])
df_online_trade['body'] = remove_num(df_online_trade['body'])
df_online_trade['body'] = remove_extra_spaces(df_online_trade['body'])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [ ]:
df_meetings['body'] = preprocess_body(df_meetings['body'])
df_meetings['body'] = remove_forwarded_message(df_meetings['body'])
df_meetings['body'] = remove_email(df_meetings['body'])
df_meetings['body'] = remove_url(df_meetings['body'])
df_meetings['body'] = remove_repeating_words(df_meetings['body'])
df_meetings['body'] = change_contracted_words(df_meetings['body'])
df_meetings['body'] = remove_spec_characters(df_meetings['body'])
df_meetings['body'] = remove_num(df_meetings['body'])
df_meetings['body'] = remove_extra_spaces(df_meetings['body'])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [ ]:
df_sent['body'] = preprocess_body(df_sent['body'])
df_sent['body'] = remove_forwarded_message(df_sent['body'])
df_sent['body'] = remove_email(df_sent['body'])
df_sent['body'] = remove_url(df_sent['body'])
df_sent['body'] = remove_repeating_words(df_sent['body'])
df_sent['body'] = change_contracted_words(df_sent['body'])
df_sent['body'] = remove_spec_characters(df_sent['body'])
df_sent['body'] = remove_num(df_sent['body'])
df_sent['body'] = remove_extra_spaces(df_sent['body'])

In [ ]:
df_corporate['body'] = preprocess_body(df_corporate['body'])
df_corporate['body'] = remove_forwarded_message(df_corporate['body'])
df_corporate['body'] = remove_email(df_corporate['body'])
df_corporate['body'] = remove_url(df_corporate['body'])
df_corporate['body'] = remove_repeating_words(df_corporate['body'])
df_corporate['body'] = change_contracted_words(df_corporate['body'])
df_corporate['body'] = remove_spec_characters(df_corporate['body'])
df_corporate['body'] = remove_num(df_corporate['body'])
df_corporate['body'] = remove_extra_spaces(df_corporate['body'])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [ ]:
df_sent_mail['body'] = preprocess_body(df_sent_mail['body'])
df_sent_mail['body'] = remove_forwarded_message(df_sent_mail['body'])
df_sent_mail['body'] = remove_email(df_sent_mail['body'])
df_sent_mail['body'] = remove_url(df_sent_mail['body'])
df_sent_mail['body'] = remove_repeating_words(df_sent_mail['body'])
df_sent_mail['body'] = change_contracted_words(df_sent_mail['body'])
df_sent_mail['body'] = remove_spec_characters(df_sent_mail['body'])
df_sent_mail['body'] = remove_num(df_sent_mail['body'])
df_sent_mail['body'] = remove_extra_spaces(df_sent_mail['body'])

In [ ]:
df_projects['body'] = preprocess_body(df_projects['body'])
df_projects['body'] = remove_forwarded_message(df_projects['body'])
df_projects['body'] = remove_email(df_projects['body'])
df_projects['body'] = remove_url(df_projects['body'])
df_projects['body'] = remove_repeating_words(df_projects['body'])
df_projects['body'] = change_contracted_words(df_projects['body'])
df_projects['body'] = remove_spec_characters(df_projects['body'])
df_projects['body'] = remove_num(df_projects['body'])
df_projects['body'] = remove_extra_spaces(df_projects['body'])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [ ]:
df_try = pd.concat([df_logistics,df_meetings], ignore_index=True)
df_try = pd.concat([df_try,df_online_trade],ignore_index=True)
df_try = pd.concat([df_try,df_sent],ignore_index=True)
df_try = pd.concat([df_try,df_sent_mail],ignore_index=True)
df_try = pd.concat([df_try,df_projects],ignore_index=True)
df_try = pd.concat([df_try,df_corporate],ignore_index=True)

In [ ]:
df_try.shape

(10526, 5)

In [ ]:
df_main = df_try.copy(deep = True)

In [ ]:
df_main.shape

(10526, 5)

In [ ]:
df_main.head()

,body
0,Here is our forecast\n\n
1,Traveling to have a business meeting takes the...
2,test successful. way to go!!!
3,"Randy,\n\n Can you send me a schedule of the s..."
4,Let's shoot for Tuesday at 11:45.


In [ ]:
df_main.body[2477]

In [ ]:
df.body[2477]

'Richard,\n\n Compare your california production to the numbers in the 2000 California Gas \nReport.  It shows 410.  But again that might be just what the two utilities \nreceive.'

In [ ]:
col = ['Subject','Folder','From','To']
df_main = df_main.drop(columns = col, axis = 1)

In [ ]:
def body_tokenize(body):
  columns = []
  for file in body:
    columns.append(word_tokenize(file))
  return columns

In [ ]:
df_main['body_tokens'] = body_tokenize(df_main['body'])
data_words = df_main.body_tokens. values.tolist()

In [ ]:
data_words = df_main.body_tokens. values.tolist()

In [ ]:
data_words

In [ ]:
bigram = Phrases(data_words, min_count=5, threshold=100)
trigram = Phrases(bigram[data_words], threshold=100)

In [ ]:
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

In [ ]:
print(trigram_mod[bigram_mod[data_words[1653]]])

In [ ]:
#Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
                    if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc 
                                        if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en', disable=['parser', 'ner'])

In [ ]:
stop_words = set(stopwords.words('english'))
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)
data_lemmatized = lemmatization(data_words_bigrams, 
                                allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
print(data_lemmatized[1653])

['event', 'description', 'date']


In [ ]:
# create dictionary and corpus both are needed for (LDA) topic modeling

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
#!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

In [ ]:
#!unzip mallet-2.0.8.zip

In [ ]:
mallet_path = './mallet-2.0.8/bin/mallet'

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# topic modeling
# corpus, dictionary and number of topics required for LDA
# alpha and eta are hyperparameters that affect sparsity of the topics
# chunksize is the number of documents to be used in each training chunk
# update_every determines how often the model parameters should be updated
# passes is the total number of training passes
# Print the Keyword in the 10 topics

In [ ]:
print(lda_model.print_topics())# The weights reflect how important a keyword is to that topic.

[(0, '0.025*"people" + 0.020*"keep" + 0.017*"move" + 0.017*"way" + 0.016*"world" + 0.015*"site" + 0.014*"learn" + 0.014*"facility" + 0.014*"manage" + 0.014*"important"'), (1, '0.041*"would" + 0.027*"thank" + 0.024*"know" + 0.024*"need" + 0.023*"question" + 0.022*"work" + 0.020*"help" + 0.018*"call" + 0.017*"make" + 0.017*"want"'), (2, '0.102*"employee" + 0.076*"time" + 0.034*"meeting" + 0.028*"meet" + 0.028*"schedule" + 0.023*"volunteer" + 0.023*"date" + 0.016*"plan" + 0.016*"participate" + 0.015*"attend"'), (3, '0.038*"year" + 0.030*"continue" + 0.023*"report" + 0.019*"support" + 0.018*"model" + 0.015*"risk" + 0.015*"process" + 0.014*"effort" + 0.014*"benefit" + 0.013*"team"'), (4, '0.037*"price" + 0.030*"deal" + 0.026*"day" + 0.026*"contract" + 0.026*"gas" + 0.024*"ticket" + 0.022*"month" + 0.021*"run" + 0.020*"unit" + 0.019*"pay"'), (5, '0.020*"go" + 0.017*"come" + 0.016*"well" + 0.014*"year" + 0.011*"enjoy" + 0.011*"game" + 0.011*"child" + 0.010*"feel" + 0.009*"fax" + 0.009*"draft"

In [ ]:
topics = lda_model.print_topics()

In [ ]:
topics[0]
# 1 Project management

(0,
 '0.025*"people" + 0.020*"keep" + 0.017*"move" + 0.017*"way" + 0.016*"world" + 0.015*"site" + 0.014*"learn" + 0.014*"facility" + 0.014*"manage" + 0.014*"important"')

In [ ]:
topics[1]
# 2 Employee appreciation/query

(1,
 '0.041*"would" + 0.027*"thank" + 0.024*"know" + 0.024*"need" + 0.023*"question" + 0.022*"work" + 0.020*"help" + 0.018*"call" + 0.017*"make" + 0.017*"want"')

In [ ]:
topics[2]
# 3 Meeting plan

(2,
 '0.102*"employee" + 0.076*"time" + 0.034*"meeting" + 0.028*"meet" + 0.028*"schedule" + 0.023*"volunteer" + 0.023*"date" + 0.016*"plan" + 0.016*"participate" + 0.015*"attend"')

In [ ]:
topics[3]
# 4 Proposistion analysis

(3,
 '0.038*"year" + 0.030*"continue" + 0.023*"report" + 0.019*"support" + 0.018*"model" + 0.015*"risk" + 0.015*"process" + 0.014*"effort" + 0.014*"benefit" + 0.013*"team"')

In [ ]:
topics[4]
# 5 Gas service request

(4,
 '0.037*"price" + 0.030*"deal" + 0.026*"day" + 0.026*"contract" + 0.026*"gas" + 0.024*"ticket" + 0.022*"month" + 0.021*"run" + 0.020*"unit" + 0.019*"pay"')

In [ ]:
topics[5]
# 6 Entertainment 

(5,
 '0.020*"go" + 0.017*"come" + 0.016*"well" + 0.014*"year" + 0.011*"enjoy" + 0.011*"game" + 0.011*"child" + 0.010*"feel" + 0.009*"fax" + 0.009*"draft"')

In [ ]:
topics[6]
# 7 Bank Offers

(6,
 '0.033*"cost" + 0.023*"value" + 0.022*"free" + 0.021*"state" + 0.020*"high" + 0.020*"rate" + 0.019*"increase" + 0.017*"plant" + 0.017*"bring" + 0.016*"expect"')

In [ ]:
topics[7]
# 8 Analysing a office event

(7,
 '0.025*"information" + 0.015*"business" + 0.013*"include" + 0.012*"contact" + 0.012*"new" + 0.012*"available" + 0.012*"group" + 0.011*"provide" + 0.011*"good" + 0.010*"follow"')

In [ ]:
topics[8]
#  9 Week/Month/Year meeting

(8,
 '0.053*"power" + 0.053*"event" + 0.028*"family" + 0.023*"build" + 0.022*"become" + 0.022*"consider" + 0.018*"director" + 0.016*"product" + 0.016*"building" + 0.016*"season"')

In [ ]:
topics[9]
# 10 Exchange Market

(9,
 '0.068*"company" + 0.044*"market" + 0.027*"say" + 0.026*"service" + 0.024*"stock" + 0.023*"energy" + 0.022*"customer" + 0.022*"dynegy" + 0.021*"offer" + 0.016*"right"')

In [ ]:
# Model perplexity and topic coherence provide a convenient
# measure to judge how good a given topic model is.
# Compute Perplexity
# a measure of how good the model is. lower the better.
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  


Perplexity:  -7.880528126307022


In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, 
                                     dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3561569551140615


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook(sort=True)
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
pyLDAvis.display(vis)

In [ ]:
id2word = corpora.Dictionary(data_lemmatized)
id2word.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=id2word, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=id2word, passes=2, workers=4)

In [ ]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.005*"time" + 0.005*"know" + 0.005*"would" + 0.005*"go" + 0.004*"call" + 0.004*"thank" + 0.004*"work" + 0.004*"get" + 0.004*"let" + 0.004*"make"
Topic: 1 Word: 0.035*"time" + 0.025*"pm" + 0.022*"date" + 0.007*"call" + 0.007*"weekly_priority" + 0.006*"thank" + 0.006*"check" + 0.006*"meeting" + 0.006*"room" + 0.005*"training"
Topic: 2 Word: 0.011*"thank" + 0.006*"help" + 0.005*"would" + 0.005*"meeting" + 0.005*"come" + 0.004*"see" + 0.004*"model" + 0.004*"go" + 0.004*"know" + 0.004*"gas"
Topic: 3 Word: 0.011*"attach" + 0.008*"time" + 0.006*"good" + 0.006*"look" + 0.005*"thank" + 0.005*"work" + 0.005*"message" + 0.005*"would" + 0.005*"find" + 0.005*"credit"
Topic: 4 Word: 0.074*"time" + 0.040*"date" + 0.024*"description" + 0.013*"white_detaile" + 0.012*"chairperson_stacey" + 0.011*"pm" + 0.009*"meet" + 0.008*"event" + 0.007*"reminder" + 0.007*"meeting"
Topic: 5 Word: 0.010*"time" + 0.008*"meeting" + 0.008*"deal" + 0.007*"meter" + 0.007*"thank" + 0.006*"date" + 0.006*"descr

In [ ]:
# Build the bigram and trigram models
bigram = Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[200]]])

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])